## Configuration

In [72]:
from neo4j import GraphDatabase
import json

URI = "neo4j://localhost:7687"   
USER = "neo4j"
PASSWORD = "2003nguyen"

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

In [73]:
def create_constraints():
    with driver.session() as session:
        session.run("""
        CREATE CONSTRAINT entity_name_unique IF NOT EXISTS
        FOR (e:Entity)
        REQUIRE e.name IS UNIQUE
        """)

create_constraints()

In [74]:
def create_triple(tx, subject, predicate, obj, source):
    tx.run(
        """
        MERGE (s:Entity {name: $subject})
        MERGE (o:Entity {name: $object})
        MERGE (s)-[r:REL {predicate: $predicate}]->(o)
        ON CREATE SET r.sources = [$source]
        ON MATCH SET 
            r.sources = CASE 
                WHEN r.sources IS NULL THEN [$source]
                WHEN NOT $source IN r.sources THEN r.sources + $source
                ELSE r.sources
            END
        """,
        subject=subject,
        predicate=predicate,
        object=obj,
        source=source,
    )

In [75]:
def load_triples_from_json(json_path: str):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    stages = data.get("stages", [])

    with driver.session() as session:
        for stage in stages:
            triples = stage.get("triples", [])
            for t in triples:
                subject = t.get("subject")
                predicate = t.get("predicate")
                obj = t.get("object")
                source = t.get("metadata", {}).get("source", "")

                if not subject or not obj:
                    continue

                session.execute_write(
                    create_triple,
                    subject,
                    predicate,
                    obj,
                    source,
                )




In [76]:
create_constraints()
load_triples_from_json("/Users/huynhnguyen/WorkDir/bachoc_1/output/nguyen-van-tuan.json")
driver.close()